In [6]:
import math
from openpyxl import load_workbook
from unipath import Path
import os
from django.core import exceptions

base = Path('d:\\', 'Users', 'javier', 'tmp')
r06 = Path(base, 'mac_06.xlsx')
r07 = Path(base, 'mac_07.xlsx')
r07b = Path(base, 'mac_07b.xlsx')
# wb6 = load_workbook(filename = r06)
# wb7 = load_workbook(filename = r07)
# wb7b = load_workbook(filename = r07b)
# ws6 = wb6['2021_06']
# ws7 = wb7['2021_06']
# ws7b = wb7b['2021_06']

In [7]:
def get_int(celda):
    """Convierte el valor de una celda en entero"""
    if celda.value == '':
        return 0
    try:
        if celda.ctype != 2:
            return 0
        return math.ceil(celda.value)
    except ValueError:
        return 0


def get_float(celda):
    """"Convierte el valor de una celda en un número flotante"""
    # valor = 0
    try:
        valor = float(celda.value)
    except ValueError:
        valor = 0
    return valor

In [40]:
def get_data_excel(file):
    try: 
        wb = load_workbook(filename = file, data_only=True)
    except Exception: 
        raise BadRequest(e)
    ws = wb.active
    remesa = ws['J7'].value[8:15]
    observaciones = ws['A41'].value
    macs = {}
    
    for row in range(9, 22):
        mac = ws[row]
        try:
            _mac = str(mac[0].value)
            if _mac[:3] == '290':
                macs[_mac] = {
                    'distrito': _mac[2:4],
                    'type': mac[1].value,
                    'days_worked': mac[2].value,
                    'hours_worked': mac[3].value,
                    'setup': mac[4].value,
                    'tramites_aplicados': mac[5].value,
                    'cards_by_update': mac[6].value,
                    'total_atenciones': mac[7].value,
                    'numbers_by_day': mac[8].value,
                    'numbers_by_machine_by_day': mac[9].value,
                    'credenciales_recibidas': mac[11].value
                }
        except ValueError:
            pass
    
    return remesa, observaciones, macs

r, o, macs = get_data_excel(r07b)

In [41]:
print(r)

2021_07


In [3]:
import os
import django
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()
INICIO = '2021-01-18'
FINAL = '2021-12-31'

from apps.productividad.models import Reporte
reportes = Reporte.objects.filter(fecha_corte__gte=INICIO, fecha_corte__lte=FINAL).order_by('fecha_corte')
reportes[0].remesa

'2021-06'

In [4]:
from apps.productividad.models import Numbers

tramites = Numbers.objects\
            .filter(reporte_semanal__fecha_corte__year=2021)\
            .values('distrito')\
            .order_by('distrito')\
            .annotate(suma_modulo=Sum('tramites_aplicados'))

pronostico = PronosticoTramites.objects.all().filter().filter(year=2021)
chart_data = []
for _distrito in ('01', '02', '03'):
    dlist = [_distrito]
    _tramites = 0
    _pronostico = 0
    for _cifras in tramites:
        if _cifras['distrito'] == _distrito:
            _tramites = _cifras['suma_modulo']
            dlist.append(_tramites)
    for _pt in pronostico:
        if f'0{_pt.distrito}' == _distrito:
            _pronostico = _pt.tramites
            dlist.append(_pronostico)
    dlist.append(_pronostico - _tramites)
    try:
        dlist.append((_tramites / _pronostico) * 100)
    except ZeroDivisionError:
        dlist.append(0)
    chart_data.append(dlist)

In [9]:
sum(r[2] for r in chart_data) - sum(r[1] for r in chart_data)
sum(r[2] for r in chart_data)

-16051

In [11]:
chart_data

[['01', 5860, -5860, 0], ['02', 6294, -6294, 0], ['03', 3897, -3897, 0]]

In [12]:
5860+6294+3897

16051

In [13]:
estatal = {
            'tramites': sum(r[1] for r in chart_data),
            'faltantes': sum(r[2] for r in chart_data) - sum(r[1] for r in chart_data),
            'pronostico': sum(r[2] for r in chart_data),
            'porcentaje': (sum(r[1] for r in chart_data) / sum(r[2] for r in chart_data)) * 100
        }

In [14]:
estatal

{'tramites': 16051,
 'faltantes': -32102,
 'pronostico': -16051,
 'porcentaje': -100.0}

In [16]:
len(pronostico)

0